In [4]:
"""
2022
For Friktion Labs
Done by: MordantBlack (enfamil#3658)
"""

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import schedule
import time
import threading
import glob
import os.path
from pycoingecko import CoinGeckoAPI

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

In [18]:
query_date = '"2022-02-15", "2022-03-15"'

In [25]:
query = """
query{
solana(network: solana) {
transfers(
  date: {between: ["""+ query_date +"""]}
  transferType: {is: transfer}
      any: [{senderAddress: {in: [
        "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
        "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
        "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
        "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
        "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
        "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
        "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
        "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
        "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
        "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
        "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM"]}}]
) {
  amount
  currency {
    symbol
  }
  date {
    date
  }
  transaction {
    signer
  }
  sender {
    address
  }
}
}
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['transfers'])
# relabel columns
df = df.rename(columns={"date.date": "date", "currency.symbol": "asset", "transaction.signer":"signer","sender.address":"sender"})
# IMPORTANT - if transaction signer & receiver is diff, drop row
df = df.query("signer == sender").reset_index(drop=True)

df.to_csv("volt01_master.csv", mode='a', header=False)
print("Master sheet created with dates Updated!") 
# read csv
df_uu = pd.read_csv("volt01_master.csv", index_col=0)
df_ts = pd.read_csv("volt01_master.csv", index_col=0)

#---------------- UNIQUE USERS ----------------------------
df_uu = df_uu.groupby(['asset', 'date']).size().reset_index(name='count')
# remove duplicated rows if any
df_uu.drop_duplicates(keep='first',inplace=True)
# save Unique Users df to csv
df_uu.to_csv("volt01_uu.csv")
print("volt01_uu.csv updated with date range " + query_date)

#---------------- TRANSACTION SIZE ----------------------------
# sum up amount according to sender addresses by asset grp
agg_functions = {'amount': 'sum'}
df_ts = df_ts.groupby(['asset','sender']).agg(agg_functions).reset_index()
# get average size of each asset grp
agg_functions_2 = {'amount': 'mean'}
df_ts = df_ts.groupby(['asset']).agg(agg_functions_2).reset_index()

agg_functions = {'count': 'sum'}
df_uu = df_uu.groupby(['asset']).agg(agg_functions).reset_index()
df_merge = pd.merge(df_ts, df_uu, on="asset")

# get asset prices
cg = CoinGeckoAPI()
# SOL, BTC, mSOL, ETH, FTT, SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"
asset_price = cg.get_price(ids='solana, bitcoin, msol, ethereum, ftx-token, serum, mango-markets, socean-staked-sol, saber, terra-luna, raydium', vs_currencies='usd')
df_price = pd.DataFrame(asset_price)
df_price = df_price.rename(columns={"solana": "SOL", "bitcoin": "BTC", "msol": "mSOL", 
                     "ethereum": "ETH", "ftx-token": "FTT", "serum": "SRM", 
                     "mango-markets": "MNGO", "socean-staked-sol": "scnSOL", 
                     "saber": "SBR", "terra-luna": "LUNA", "raydium": "RAY"})
df_price = df_price.T
df_price = df_price.rename_axis('asset').reset_index()
# Concatenate 2 tables tgt aligning assets
final_df = pd.merge(df_merge, df_price, on='asset')
# Get average transaction size in USD
final_df["tx_size"] = final_df["amount"] * final_df["usd"]
final_df.sort_values(by=['tx_size'], inplace=True)
final_df = final_df.fillna(0)
# save Tx Size df to csv
final_df.to_csv("volt01_ts.csv")
print("volt01_ts.csv updated with date range " + query_date)

Master sheet created with dates Updated!
volt01_uu.csv updated with date range "2022-02-15", "2022-03-15"
volt01_ts.csv updated with date range "2022-02-15", "2022-03-15"


In [ ]:
# Will need to manually keep in date range under query_date if run into errors
extract_volt_data("volt01_master.csv")

In [ ]:
extract_volt_data("volt01_master_wd.csv")

In [ ]:
extract_volt_data("volt02_master.csv")

In [ ]:
extract_volt_data("volt02_master_wd.csv")

In [ ]:
# If prefer to set a schedule for updating, below will run everyday at a certain time of your specification. 
# Leaving 2 minutes inbetween each function 
# when using .do, argument within function is placed in the following manner
schedule.every().day.at("12:04").do(extract_volt_data, "volt01_master.csv")
schedule.every().day.at("12:06").do(extract_volt_data, "volt01_master_wd.csv")
schedule.every().day.at("12:08").do(extract_volt_data, "volt02_master.csv")
schedule.every().day.at("12:10").do(extract_volt_data, "volt02_master_wd.csv")

while True:
    # Checks whether a scheduled task 
    # is pending to run or not
    schedule.run_pending()
    time.sleep(1)